# 使用keras实现卷积神经网络
这里仍然使用mnist数据   
通过tf.keras.Model来自定义网络结构

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import data as tfdata
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow import losses
from tensorflow.keras import initializers as init

from tensorflow.keras.utils import plot_model

In [3]:
class MNistLoader():
    def __init__(self):
        # 定义mnist对象
        mnist = tf.keras.datasets.mnist
        # 读取mnist数据
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # mnist的数据为 28*28的0~255的值, 需要将图像转化为[张数， 宽度， 高度， 通道]， 并归一化
        self.train_data = np.expand_dims(self.train_data.astype(np.float32)/255.0, axis=-1)
        self.test_data = np.expand_dims(self.test_data.astype(np.float32)/255.0, axis=-1)

        self.train_label = self.train_label.astype(np.float32)
        self.test_label = self.test_label.astype(np.float32)

        # 统计相关信息
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]
    
    def get_batch(self, batch_size):
        # 从train_data中随机抽出batch_size个数据
        index = np.random.randint(0, self.num_train_data, batch_size)
        # np数组可以接受list索引，返回list索引对应的值, 原生数据不支持
        return self.train_data[index, :], self.train_label[index]
        


## 定义CNN模型
和MLP相比，只是新加入了卷积层和池化层，这里的网络结构并不唯一，可以增加、删除、调整CNN的网络结构和参数，以达到更好的效果

In [7]:
class CNN(tf.keras.Model):
    def __init__(self):
        # 继承父类的属性和方法
        super().__init__()
        # 定义网络层
        self.conv1 = layers.Conv2D(
            # 卷积层神经元（卷积核）数目, 默认使用kernel_initializer默认使用glorot_uniform进行初始化，简单来讲就是有32个不一样的卷积核，可以指定kernel_initializer
            filters=32,
            # 感受野大小
            kernel_size=[5, 5],
            # padding策略, same/vaild, same表示图片卷积后大小不变，valid表示卷积后图像减小（不对标远进行填充）
            padding='same',
            # 激活函数
            activation=tf.nn.relu
        )

        self.pool1 = layers.MaxPool2D(
            pool_size=[2,2],
            strides=2
        )

        self.conv2 = layers.Conv2D(
            filters=64,
            kernel_size=[5,5],
            padding="same",
            activation=tf.nn.relu
        )

        self.pool2 = layers.MaxPool2D(pool_size=[2,2], strides=2)
        # TODO, 和Flatten的区别是什么
        self.flatten = layers.Reshape(target_shape=(7*7*64, ))
        self.dense1 = layers.Dense(units=1024, activation=tf.nn.relu)
        self.dense2 = layers.Dense(units=10)
    
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        output = tf.nn.softmax(x)
        return output

## 定义超参和实例化对象


In [8]:
# 定义超参
batch_size = 50
epoch = 5
learning_rate = 0.001

# 实例化数据和模型对象
mnist = MNistLoader()
cnn = CNN()

# 
optimizer = optimizers.Adam(learning_rate=learning_rate)

## 训练过程

In [ ]:
# num_batches 
num_batches = int(mnist.num_train_data*epoch//batch_size)
